In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import faiss
import numpy as np

# Przykładowe dane
documents = [
    "Aby poprawnie wykonać przysiad, upewnij się, że twoje stopy są rozstawione na szerokość barków.",
    "Podczas wyciskania sztangi na ławce poziomej, upewnij się, że twoje łopatki są ściągnięte.",
    # Dodaj więcej dokumentów dotyczących ćwiczeń...
]

# Tokenizacja i wektoryzacja
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embeddings = []
for doc in documents:
    inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
    embeddings.append(inputs['input_ids'].mean(dim=1).detach().numpy())

# Utworzenie indexu FAISS
index = faiss.IndexFlatL2(embeddings[0].shape[1])
index.add(np.vstack(embeddings))

# Model generacyjny
generator = pipeline("text2text-generation", model="facebook/bart-large-cnn")

def retrieve(query):
    query_embedding = tokenizer(query, return_tensors="pt", truncation=True, padding=True)['input_ids'].mean(dim=1).detach().numpy()
    D, I = index.search(query_embedding, k=1)
    return documents[I[0][0]]

def generate_answer(query):
    retrieved_doc = retrieve(query)
    input_text = f"Query: {query}\nDocument: {retrieved_doc}\nAnswer:"
    generated = generator(input_text, max_length=50, clean_up_tokenization_spaces=True)
    return generated[0]['generated_text']

# Przykładowe pytanie
question = "Jak poprawnie wykonywać przysiady?"
answer = generate_answer(question)
print(answer)


C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'faiss'